<a href="https://colab.research.google.com/github/LauraMartinus/ukuxhumana/blob/master/fairseq/comparing_decomposers/ts_bpe_vs_morfessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pytorch

In [1]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x5885c000 @  0x7f739f24a2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


## Fairseq


In [2]:
!git clone https://github.com/pytorch/fairseq.git
import os
os.chdir("fairseq/")
!pip install -r requirements.txt
%run -i 'setup.py' build develop

Cloning into 'fairseq'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 3094 (delta 13), reused 8 (delta 7), pack-reused 3063
Receiving objects: 100% (3094/3094), 3.14 MiB | 8.81 MiB/s, done.
Resolving deltas: 100% (2247/2247), done.
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/fairseq
copying fairseq/sequence_scorer.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/options.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/sequence_generator.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/distributed_utils.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/__init__.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/legacy_distributed_data_parallel.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/search.py -> build/lib.linux-x86_64-3.6/fairseq
copying fairseq/multiprocessi

## Subword

In [3]:
!git clone https://github.com/rsennrich/subword-nmt

Cloning into 'subword-nmt'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 485 (delta 0), reused 1 (delta 0), pack-reused 481
Receiving objects: 100% (485/485), 205.64 KiB | 1.09 MiB/s, done.
Resolving deltas: 100% (287/287), done.


## Data

In [4]:
!git clone https://github.com/LauraMartinus/ukuxhumana.git

Cloning into 'ukuxhumana'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 1282 (delta 4), reused 4 (delta 0), pack-reused 1266
Receiving objects: 100% (1282/1282), 381.24 MiB | 21.61 MiB/s, done.
Resolving deltas: 100% (611/611), done.
Checking out files: 100% (300/300), done.


## BPE

In [0]:
os.chdir('../')
!ls

fairseq  sample_data


In [0]:
#os.chdir('../')
#os.chdir('fairseq/')
!cat fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > combine.txt

!fairseq/subword-nmt/learn_bpe.py -s 8000 <combine.txt> ents.codes
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.train.ts > train.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.dev.ts > valid.ts
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c ents.codes < fairseq/ukuxhumana/clean/en_ts/ents_parallel.test.ts > test.ts

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang ts --target-lang en --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/ents

!mkdir -p checkpoint/fconv
!python train.py data-bin/ents \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 2000 \
    --arch fconv --save-dir checkpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/ents --path checkpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang ts --target-lang en

## Morfessor

In [0]:
##NEED TO CHANGE FILE NAMES
TEXT=".."
%run 'preprocess.py' --source-lang nso --target-lang en --trainpref $TEXT/ennso_parallel.train.decomp --validpref $TEXT/ennso_parallel.dev.decomp --testpref $TEXT/ennso_parallel.test.decomp --destdir data-bin/de_ennso

!mkdir -p checkpoint/fconv
!python train.py data-bin/de_ennso \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 500 \
    --arch fconv --save-dir chcpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/de_ennso --path chcpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang nso --target-lang en

## BPE Eng Morfessor NSO

In [0]:
os.chdir('../')
#!cat enzu_parallel.train.en enzu_parallel.train.zu > combine.txt

!fairseq/subword-nmt/learn_bpe.py -s 8000 <ennso_parallel.train.en> ennso.codes
!fairseq/subword-nmt/apply_bpe.py -c ennso.codes < ennso_parallel.train.en > ennso_parallel.train.decomp.en
#!fairseq/subword-nmt/apply_bpe.py -c enzu.codes < enzu_parallel.train.zu > train.zu
!fairseq/subword-nmt/apply_bpe.py -c ennso.codes < ennso_parallel.dev.en > ennso_parallel.dev.decomp.en
#!fairseq/subword-nmt/apply_bpe.py -c enzu.codes < enzu_parallel.dev.zu > valid.zu
!fairseq/subword-nmt/apply_bpe.py -c ennso.codes < ennso_parallel.test.en > ennso_parallel.test.decomp.en
#!fairseq/subword-nmt/apply_bpe.py -c enzu.codes < enzu_parallel.test.zu > test.zu
os.chdir('fairseq/')

In [0]:
TEXT=".."
%run 'preprocess.py' --source-lang nso --target-lang en --trainpref $TEXT/ennso_parallel.train.decomp --validpref $TEXT/ennso_parallel.dev.decomp --testpref $TEXT/ennso_parallel.test.decomp --destdir data-bin/de_ennso

!mkdir -p checkpoint/fconv
!python train.py data-bin/de_ennso \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 500 \
    --arch fconv --save-dir chkpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/de_ennso --path chkpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang nso --target-lang en

## Clean

In [5]:
!ls

build		      fairseq		multiprocessing_train.py  scripts
CONTRIBUTING.md       fairseq.egg-info	PATENTS			  setup.py
distributed_train.py  fairseq.gif	preprocess.py		  subword-nmt
docs		      generate.py	README.md		  tests
eval_lm.py	      interactive.py	requirements.txt	  train.py
examples	      LICENSE		score.py		  ukuxhumana


In [7]:
TEXT="ukuxhumana/"
%run 'preprocess.py' --source-lang ts --target-lang en --trainpref $TEXT/clean/en_ts/ents_parallel.train --validpref $TEXT/clean/en_ts/ents_parallel.dev --testpref $TEXT/clean/en_ts/ents_parallel.test --destdir data-bin/de_ents

!mkdir -p checkpoint/fconv
!python train.py data-bin/de_ents \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 2000 \
    --arch fconv --save-dir chckpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/de_ents --path chckpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang ts --target-lang en

Namespace(alignfile=None, destdir='data-bin/de_ents', joined_dictionary=False, nwordssrc=-1, nwordstgt=-1, only_source=False, output_format='binary', padding_factor=8, source_lang='ts', srcdict=None, target_lang='en', testpref='ukuxhumana//clean/en_ts/ents_parallel.test', tgtdict=None, thresholdsrc=0, thresholdtgt=0, trainpref='ukuxhumana//clean/en_ts/ents_parallel.train', validpref='ukuxhumana//clean/en_ts/ents_parallel.dev', workers=1)
| [ts] Dictionary: 42471 types
| [ts] ukuxhumana//clean/en_ts/ents_parallel.train.ts: 135510 sents, 1757400 tokens, 0.0% replaced by <unk>
| [ts] Dictionary: 42471 types
| [ts] ukuxhumana//clean/en_ts/ents_parallel.dev.ts: 55077 sents, 729219 tokens, 1.1% replaced by <unk>
| [ts] Dictionary: 42471 types
| [ts] ukuxhumana//clean/en_ts/ents_parallel.test.ts: 3000 sents, 39681 tokens, 1.07% replaced by <unk>
| [en] Dictionary: 35271 types
| [en] ukuxhumana//clean/en_ts/ents_parallel.train.en: 135510 sents, 1509785 tokens, 0.0% replaced by <unk>
| [en] Dic

## Results

BPE 8k: 36.11

Morfessor: 

BPE Eng Morfessor Ts: 

Clean: 39.19